In [ ]:
import os
import sys
import django
import pprint
import pandas as pd
# from django_pandas.io import read_frame
import itertools
from blocks import block
import numpy as np

# setup the djhango settings file.  Change this to use the settings file that connects you to your desired database
os.environ["DJANGO_SETTINGS_MODULE"] = "djangochem.settings.denn"
# this must be run to setup access to the django settings and make database access work etc.
django.setup()

# import the models that you want to access
from pgmols.models import Mol, Calc, Geom
from jobs.models import Job, JobConfig

# this is all setup for the notebook
from IPython.display import HTML
import matplotlib
%matplotlib inline
from rdkit.Chem import AllChem as Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools # headsup: this import change the behavior of dataframes with mols in them
# some global configuration of the pandastools
PandasTools.molRepresentation = 'svg'
PandasTools.molSize = (200,200)

#constatns
HA_TO_EV = 27.211399
# this is a little helper function to render images inside a dataframe
# once again, there are ways to monkey patch the rendering of dataframes, but I am trying to 
# avoid most of that to make things a bit easier to understand

def show(df):
    return HTML(df.to_html(escape=False))

In [ ]:
# exprt_list=['BIOPPFDHKHWJIA-UHFFFAOYSA-N','NXXNVJDXUHMAHU-UHFFFAOYSA-N','QQLRAWFMOWCSBU-UHFFFAOYSA-N']

In [ ]:
#import simplejson as simplejson


from django.db.models.expressions import RawSQL
from django.core import serializers
from django.db.models import Q,F,Min,Case,When,Value
#parentjob__config__name__contains='tddft',
all_calcs = Calc.objects \
    .filter(Q(mol__group__name__exact='sf'),
#            Q(mol__inchikey='BIOPPFDHKHWJIA-UHFFFAOYSA-N')|Q(mol__inchikey='NXXNVJDXUHMAHU-UHFFFAOYSA-N')|Q(mol__inchikey='QQLRAWFMOWCSBU-UHFFFAOYSA-N'),
            parentjob__status__exact='done',
            method__name__contains='hybrid') \
    .exclude(Q(method__name__contains='_tda_')
               |
#              Q(geoms__parentjob__config__parent_class_name__exact='Mol')|
              Q(mol__inchikey__exact='STTGYIUESPWXOW-UHFFFAOYNA-N')
            )\
    .annotate(totalenergy=RawSQL("((props->>%s)::numeric)", ('totalenergy',)))\
    .annotate(s2=Case(When(Q(parentjob__config__name='b3lyp_6-31gs_opt_t1_qchem')|
                                 Q(parentjob__config__name='b3lyp_6-31gs_opt_BS_qchem'),
                                 then=RawSQL("((props->>%s)::numeric)", ('s2',)))),default=Value(None))\
    .annotate(excitedstates=Case(When(parentjob__config__name__contains='tddft',
                                 then=RawSQL("((props->>%s)::json)", ('excitedstates',)))), default=Value(None))\
    .order_by('mol__inchikey','parentjob__config__name','totalenergy')

    # .annotate(t1_opt=Case(When(=parentjob__config__name='b3lyp_6-31gs_opt_t1_qchem',
    #                              properties__method__name='b3lyp_tddft_631gs_rpa_s0_geom',
    #                              then='properties__value')),default=Value(None))\
    # .annotate(ene=Case(When(parentjob__config__name__contains='tddft',
    #                              then=RawSQL("((props#>%s)::json)", ('excitedstates',)))), default=Value(None))\


all_calcs_values=all_calcs.values('id',
#                                  'geoms',
                                  'geoms__parents',
#                                  'geoms__parents__parents',
                                  'geoms__parents__xyz',
                                  'geoms__method',
                                  'geoms__xyz',
                                  'geoms__parentjob__config__name',
                                  'mol__inchikey',
                                  'mol__smiles',
                                  'mol__tags',
                                  'method__name',
                                  'parentjob__config__name',
                                  'totalenergy',
                                  's2',
                                  'excitedstates')
#print(all_calcs_values.query)
#print(list(all_calcs.values_list('totalenergy')))
all_calcs_dict=list(all_calcs_values)
df = pd.DataFrame(all_calcs_dict)

#df=df[df.filter(df.excitedstates!=None)]
#print(simplejson.dumps(all_calcs_dict))
#df['parentjob__config__name']=df['parentjob__config__name'].str.replace('b3lyp_6-31gs_opt_qchem','b3lyp_6-31gs_opt_BS_qchem')
df['parentjob__config__name']=df['parentjob__config__name'].str.replace('-','_')

df['totalenergy']=df['totalenergy'].astype(float)
df['s2']=df['s2'].astype(float)
#df.sort_values('mol__inchikey',inplace=True)
#print(df.dtypes)
#df['totalenergy']=pd.to_numeric(df['totalenergy'],errors='coerce')
#print(df.head())
#show(df[:1])
#df_props=df.pop('props')
#df = pd.concat([df,df_props.apply(pd.Series)],axis=1)
#df['block'] = df.mol__smiles.apply(lambda s: block.Block(smiles=s))
#df['mol'] = df.block.apply(lambda b: b.mol)
#df['mol']=df.mol__smiles.apply(Chem.MolFromSmiles)
cols=['id','geoms__parents','mol__inchikey','parentjob__config__name','totalenergy','mol__tags']
#df[cols][:10]
show(df[cols][:])
df.parentjob__config__name.value_counts()


In [ ]:
def to_xyz(geoms):
    output = str(len(geoms)) + "\n\n"
    for g in geoms:
        output += " ".join([PERIODICTABLE.GetElementSymbol(int(g[0])), str(g[1]), str(g[2]), str(g[3])]) + "\n"
    return output
PERIODICTABLE = Chem.GetPeriodicTable()


#Get B3LYP geometries in XYZ file format for conformers indexed by geoms geomID
df_xyz=df[['geoms__method','geoms__parents','geoms__parentjob__config__name','mol__inchikey','geoms__xyz']]
show(df_xyz.sample(2))


In [ ]:
df_xyz=df_xyz[df_xyz.geoms__parentjob__config__name=='b3lyp_6-31gs_opt_BS_qchem']
df_xyz=df_xyz.drop_duplicates(subset=['geoms__parents','mol__inchikey'])
df_xyz['xyz']=df['geoms__xyz'].apply(to_xyz)
df_xyz=df_xyz.set_index(['geoms__parents'])

df_xyz['geoms__method'].mean()

In [ ]:
cols=['geoms__parents','mol__inchikey','parentjob__config__name','totalenergy','s2','excitedstates']
df_p=df[cols].set_index(['mol__inchikey','geoms__parents'])
#dfp=df_p.unstack('parentjob__config__name',)
#dfp=pd.pivot_table(df[cols],index=['mol__inchikey','geoms__parents'],columns='parentjob__config__name',values=['totalenergy','s2','excitedstates'],aggfunc=np.count_nonzero)
dfp=df_p.pivot(columns='parentjob__config__name')
#print(dfp[:1])
dft=dfp['totalenergy']['b3lyp_6_31gs_opt_BS_qchem']
dft=dft.groupby(level=0).min().to_frame()

df_calc=pd.DataFrame(index=dfp.index)
df_calc['t1_opt']=(dfp['totalenergy']['b3lyp_6_31gs_opt_t1_qchem']-dfp['totalenergy']['b3lyp_6_31gs_opt_BS_qchem'])*HA_TO_EV
df_calc['t1_opt1']=(dfp['totalenergy']['b3lyp_6_31gs_opt_t1_qchem']-dfp['totalenergy']['b3lyp_6_31gs_tddft'])*HA_TO_EV
df_calc['dft_energy']=(dfp['totalenergy']['b3lyp_6_31gs_opt_BS_qchem']-dft['b3lyp_6_31gs_opt_BS_qchem'])*HA_TO_EV
df_calc['s2_BS']=dfp['s2']['b3lyp_6_31gs_opt_BS_qchem']
df_calc['s2_t1']=dfp['s2']['b3lyp_6_31gs_opt_t1_qchem']
        #df_calc=pd.merge(df_calc, confs_norm_e, how='outer', right_index=True, left_index=True)
#df_calc=pd.concat([df_calc, confs_norm_e],axis=1)
df_calc.index = df_calc.index.set_names(['mol__inchikey','geoms__parents'])
#df_calc=df_calc.sort_index(by='dft_energy',axis=0)
df_calc=df_calc.groupby(level=0, group_keys=False).apply(lambda x: x.sort_values('dft_energy'))
#df_calc=df_calc.sortlevel(level=1,sort_remaining=False)
cols_calc=['t1_opt','t1_opt1']
#dfp[cols_calc][:10]
#df_calc[:30]
#dfpc=dfp[cols_calc]#.reset_index(level='parentjob__config__name',drop=True)
#print(dfp.index)
#dfp1 = pd.merge(dfp, dft, how='right', right_index=True, left_index=True)
#df_calc['conf_dft_energy']=df_calc[b3lyp_6_31gs_opt_BS_qchem_x]-confs_norm_e.totalenergy_y
#dfp1['totalenergy'][['b3lyp_6_31gs_opt_BS_qchem','b3lyp_6_31gs_tddft']][:20]
#dfpc[:1]
#df_calc['diff_energy']=df_calc['dft_energy']-df_calc['dftb_energy'].astype(float)
df_calc[:10]
#dfp[:1]

In [ ]:
#start = t.time()
dfe = dfp.excitedstates.b3lyp_6_31gs_tddft.apply(pd.Series)
dfe1 = dfe.stack()
dfe2 = dfe1.apply(pd.Series)
#dfe2.multiplicity=='singlet'
dfe2 = dfe2[['energy','multiplicity','oscillator_strength','pbht']]
#dfe2.index = dfe2.index.set_names(['mol__inchikey','geoms__parents','state'])
#dfe2 = dfe2.reset_index().set_index(['mol__inchikey','geoms__parents','multiplicity','state'])
#dfe2 = dfe2.sort_index(level='multiplicity')
#dfe2 = dfe2.sort_index(level='geoms__parents')
#dfe2.dropna(inplace=True)
#dfe3 = dfe2.pivot(columns='multiplicity')
#dfe3 = dfe2.unstack('multiplicity')
df_s = dfe2[dfe2.multiplicity=='singlet'].rename(columns={'energy':'s1_vert'})
df_t = dfe2[dfe2.multiplicity=='triplet'].rename(columns={'energy':'t1_vert'})
#dfe3.index.set_names(['mol__inchikey', 'geoms__parents', 'tran'],inplace=True)
#dfe3.xs(tran,level=[0,1])
dfs1=df_s.groupby(level=[0,1], group_keys=False).apply(lambda x: x.sort_values('s1_vert').iloc[0])
dft1=df_t.groupby(level=[0,1], group_keys=False).apply(lambda x: x.sort_values('t1_vert').iloc[0])
dft2=df_t.groupby(level=[0,1], group_keys=False).apply(lambda x: x.sort_values('t1_vert').iloc[1]).rename(columns={'t1_vert':'t2_vert'})

#dfs1=dfs1.groupby(level=[0,1]).min()['s1_vert']
#dft1=dft1.groupby(level=[0,1]).min()['t1_vert']
#show(df_s[:10])
#show(dft2[:10])
#print(dft1[:10])
df_ex = dfs1.join(dft1,lsuffix='_s1').join(dft2, lsuffix='_t1',rsuffix='_t2')
#print(dfe2.index)
#dfe2[:10]
#dfe3[:3]
#print(dfpc.index)
df_calcs = pd.merge(df_calc, df_ex, how='outer', right_index=True, left_index=True)


df_acenes = pd.read_csv('/home/denn/harvard/SF/Library/acenes_calcs.csv')
#df_acenes.set_index(['mol__inchikey','geoms__parents'],inplace=True)
df_calcs.reset_index(inplace=True)
#df_calcs = pd.merge(df_calcs, df_acenes, how='outer', right_index=True, left_index=True)
#df_acenes.set_index(['mol__inchikey','geoms__parents'],inplace=True)
df_calcs=df_calcs.append(df_acenes)
df_calcs.set_index(['mol__inchikey','geoms__parents'],inplace=True)
#show(df_calcs.ix['SLIUAWYAILUBJU-UHFFFAOYNA-N'])
df_calcs[:3]
#dfs1[:10]
#rint(t.time()-start)

In [ ]:
if 'lcwpbe_6_31gs_tddft_qchem' in dfp.excitedstates.columns:
    dfe = dfp.excitedstates.lcwpbe_6_31gs_tddft_qchem.apply(pd.Series)
    
    dfe1 = dfe.stack()
    dfe2 = dfe1.apply(pd.Series)

    dfe2 = dfe2[['energy','multiplicity','oscillator_strength','pbht']]

    df_s = dfe2[dfe2.multiplicity=='singlet'].rename(columns={'energy':'s1_vert_lcpbe'})
#     df_t = dfe2[dfe2.multiplicity=='triplet'].rename(columns={'energy':'t1_vert_lcpbe'})
    
    dfs1=df_s.groupby(level=[0,1], group_keys=False).apply(lambda x: x.sort_values('s1_vert_lcpbe').iloc[0])
#     dft1=df_t.groupby(level=[0,1], group_keys=False).apply(lambda x: x.sort_values('t1_vert_lcpbe').iloc[0])
#     dft2=df_t.groupby(level=[0,1], group_keys=False).apply(lambda x: x.sort_values('t1_vert_lcpbe').iloc[1]).rename(columns={'t1_vert_lcpbe':'t2_vert_lcpbe'})

    #df_ex = dfs1.join(dft1,lsuffix='_s1_lcpbe')#.join(dft2, lsuffix='_t1_lcpbe',rsuffix='_t2_lcpbe')
    print("lcpbe_count",dfs1.shape)
    #df_calcs = pd.merge(df_calcs, df_ex, how='outer', right_index=True, left_index=True)
    df_calcs = pd.merge(df_calcs, dfs1, how='outer', right_index=True, left_index=True,suffixes=('','_s1_wb97xd'))
show(df_calcs[:3])

In [ ]:
if 'wb97xd_6_31gs_tddft' in dfp.excitedstates.columns:
    dfe = dfp.excitedstates.wb97xd_6_31gs_tddft.apply(pd.Series)
    dfe1 = dfe.stack()
    dfe2 = dfe1.apply(pd.Series)

    dfe2 = dfe2[['energy','multiplicity','oscillator_strength','pbht']]

    df_s = dfe2[dfe2.multiplicity=='singlet'].rename(columns={'energy':'s1_vert_wb97xd'})
    df_t = dfe2[dfe2.multiplicity=='triplet'].rename(columns={'energy':'t1_vert_wb97xd'})
    
    dfs1=df_s.groupby(level=[0,1], group_keys=False).apply(lambda x: x.sort_values('s1_vert_wb97xd').iloc[0])
    
    dft1=df_t.groupby(level=[0,1], group_keys=False).apply(lambda x: x.sort_values('t1_vert_wb97xd').iloc[0])
    dft2=df_t.groupby(level=[0,1], group_keys=False).apply(lambda x: x.sort_values('t1_vert_wb97xd').iloc[1]).rename(columns={'t1_vert_wb97xd':'t2_vert_wb97xd'})
    df_ex = dfs1.join(dft1,lsuffix='_s1_wb97xd').join(dft2, lsuffix='_t1_wb97xd',rsuffix='_t2_wb97xd')
    print("wb97xd_count",dfs1.shape)
    df_calcs = pd.merge(df_calcs, df_ex, how='outer', right_index=True, left_index=True)
#     df_calcs = pd.merge(df_calcs, dfs1, how='outer', right_index=True, left_index=True,suffixes=('','_s1_wb97xd'))
show(df_calcs[:3])


In [ ]:
if 'm062x_6_31gs_tddft' in dfp.excitedstates.columns:
    dfe = dfp.excitedstates.m062x_6_31gs_tddft.apply(pd.Series)
    dfe1 = dfe.stack()
    dfe2 = dfe1.apply(pd.Series)

    dfe2 = dfe2[['energy','multiplicity','oscillator_strength','pbht']]

    df_s = dfe2[dfe2.multiplicity=='singlet'].rename(columns={'energy':'s1_vert_m062x'})
    df_t = dfe2[dfe2.multiplicity=='triplet'].rename(columns={'energy':'t1_vert_m062x'})
    
    dfs1=df_s.groupby(level=[0,1], group_keys=False).apply(lambda x: x.sort_values('s1_vert_m062x').iloc[0])
    
    dft1=df_t.groupby(level=[0,1], group_keys=False).apply(lambda x: x.sort_values('t1_vert_m062x').iloc[0])
    dft2=df_t.groupby(level=[0,1], group_keys=False).apply(lambda x: x.sort_values('t1_vert_m062x').iloc[1]).rename(columns={'t1_vert_m062x':'t2_vert_m062x'})
    df_ex = dfs1.join(dft1,lsuffix='_s1_m062x').join(dft2, lsuffix='_t1_m062x',rsuffix='_t2_m062x')
    print("m062x_count",dfs1.shape)
    df_calcs = pd.merge(df_calcs, df_ex, how='outer', right_index=True, left_index=True)
#     df_calcs = pd.merge(df_calcs, dfs1, how='outer', right_index=True, left_index=True,suffixes=('','_s1_m062x'))
show(df_calcs[:3])

In [ ]:
dfexp = pd.read_csv('/home/denn/harvard/SF/Library/exp_set_1_th_no_pentderiv.csv')
dfexp['block'] = dfexp.smiles.apply(lambda s: block.Block(smiles=s))
dfexp['mol'] = dfexp.block.apply(lambda b: b.mol)
dfexp['mol__inchikey'] = dfexp.block.apply(lambda b: b.inchikey)
dfexpi = dfexp.set_index('mol__inchikey')

df_all = pd.merge(df_calcs, dfexpi, how='right', right_index=True, left_index=True)

#df_calcs[:10]
show(df_all.ix['SLIUAWYAILUBJU-UHFFFAOYNA-N'])
#show(df_all.ix['STTGYIUESPWXOW-UHFFFAOYNA-N'])
#show(df_all[df_all.t1_opt.isnull()|df_all.t1_opt1.isnull()]['mol'].to_frame())
#print()

In [ ]:
# df_check1=pd.merge(df_all.reset_index(),df_xyz.reset_index()).reset_index(drop=True)#.join(df_smiles,how='inner')
# import pybel
# #from dask import dataframe as dd
# #pybel.informats
# #pybel.outformats
# #dd_check=dd.from_pandas(df_check,10)
# #%timeit pybel.readstring('xyz',df_check.loc['YZLGAWUIHCFDLO']['xyz']).write('inchikey')

# def obl(xyz):
#     m=pybel.readstring('xyz',xyz)
#     return pd.Series({'can':m.write('can'),'check_inchi':m.write('inchikey').split('-')[0]})

# #df_check['check_inchi']=df_check['xyz'].apply(lambda m: pybel.readstring('xyz',m).write('inchikey').split('-')[0])
# #df_check['check_smi']=df_check['xyz'].apply(lambda m: pybel.readstring('xyz',m).write('can'))
# df_check=df_check1.join(df_check1['xyz'].apply(obl))
# #dd_check['check_inchi']=dd_check['xyz'].apply(func=lambda m: pybel.readstring('xyz',m).write('inchikey'),meta=dd_check['xyz'])

# #df_check.sample(1)
# #df_check.mol__smiles
# df_not_valid=df_check[df_check.mol__inchikey!=df_check.check_inchi]
# df_not_valid_s=pd.merge(df_not_valid,df_smiles.reset_index())
# #df_not_valid_s[['mol__smiles','can']].sample(10)
# df_not_valid_s['mol']=df_not_valid_s.mol__smiles.apply(Chem.MolFromSmiles)
# #df_check.sample(1)
# #df_check.mol__smiles
# df_valid=df_check[df_check.mol__inchikey==df_check.check_inchi]
# df_valid_s=pd.merge(df_valid,df_smiles.reset_index())
# #df_not_valid_s[['mol__smiles','can']].sample(10)
# #df_not_valid_s['mol']=df_not_valid_s.mol__smiles.apply(Chem.MolFromSmiles)
# df_valid_s.count()
# df_all=df_valid.set_index(['mol__inchikey','geoms__parents'])

In [ ]:
df_high=df_all[df_all.dft_energy>0.1]
#print(df_high.count())
df_min=df_all[df_all.dft_energy==0.0]
print(df_min.count())
df_low=df_all[df_all.dft_energy<=0.1]
print(df_low.count())

In [ ]:
df_low.to_pickle('/home/denn/home2/data/SF/SF_calibraion_data_set.pkl')

In [ ]:
from rdkit.Chem import rdDepictor
from rdkit.Chem.Draw import rdMolDraw2D
def moltosvg(mol,molSize=(450,150),kekulize=True):
    mc = Chem.Mol(mol.ToBinary())
    if kekulize:
        try:
            Chem.Kekulize(mc)
        except:
            mc = Chem.Mol(mol.ToBinary())
    if not mc.GetNumConformers():
        rdDepictor.Compute2DCoords(mc)
    drawer = rdMolDraw2D.MolDraw2DSVG(molSize[0],molSize[1])
    drawer.DrawMolecule(mc)
    drawer.FinishDrawing()
    svg = drawer.GetDrawingText()
    # It seems that the svg renderer used doesn't quite hit the spec.
    # Here are some fixes to make it work in the notebook, although I think
    # the underlying issue needs to be resolved at the generation step
    return svg.replace('svg:','')

mols = df_all.mol.groupby(level=0).head(1)
#[mols.get_value(i) for i in range(mols.size)]
#mols[0]

#[moltosvg(m).replace('\n','') for m in df_all.mol.groupby(level=0).head(1)]

#df_all.mol.apply(lambda m: Chem.MolToSVG(m,0))

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot
from plotly.graph_objs import *
#from sklearn import datasets, linear_model

from numpy import arange,array,ones
from scipy import stats

def plot_calc_vs_theo(dfi:pd.DataFrame, calc, exp):
    init_notebook_mode()

    df=dfi[[exp,calc]].dropna()

    xt1=df[exp].as_matrix()
    yt1=df[calc].as_matrix()

    slope, intercept, r_value, p_value, std_err = stats.linregress(xt1,yt1)
    line = slope*xt1+intercept

    print(slope)
    print(intercept)
    
    trace0 = Scatter(
        x=xt1,
        y=yt1,
        mode='markers',
        name=calc,
    #     marker=dict(

    #     ),
        text=[i[0]+' : '+str(i[1]) for i in df.index.get_values()],
    )
    trace2 = Scatter(
                      x=xt1, 
                      y=line, 
                      mode='lines',
                      marker=Marker(color='rgb(31, 119, 180)'),
                      name='Fit'
                      )

    annotation = Annotation(
                        xref='paper',
                        yref='paper',
                       x=0,
                       y=1,
         #xanchor='left',
         #yanchor='top',
                      text='Slope: '+str(slope)+' r_value: '+str(r_value)+'\n Intercept: '+str(intercept),
                      showarrow=False,
                      font=Font(size=16)
                      )

    data = [trace0,trace2]
    layout = Layout(
        xaxis= dict(
            title= exp,
            ticklen= 5,
            zeroline= False,
            gridwidth= 1,
            
        ),
        yaxis=dict(
            title= calc,
            ticklen= 5,
            gridwidth= 1,
            
        ),
    #     shapes=dict(
    #             type='line',
    #             xref='x',
    #             yref= 'y',
    #             x0= 0,
    #             y0= 0,
    #             x1= 5,
    #             y1= 5,
    #     ),
         annotations=[annotation],
        showlegend=False,
        height=700,
        width=700,
    )

    fig = dict( data=data, layout=layout )

    iplot(fig,image_height=1200,image_width=1200)
    #plot(fig,image_height=1200,image_width=1200)


In [ ]:
def plot_calc_vs_theo_std(dfi:pd.DataFrame, calc, exp, height=800,width=800):
    init_notebook_mode()

    df=dfi[[exp,calc]].dropna()
    
    df_group=df.groupby(level=0)
    xt1=df_group[exp].mean().values
    yt1=df_group[calc].mean().values
    yt1e=df_group[calc].std().fillna(0.).values

    slope, intercept, r_value, p_value, std_err = stats.linregress(xt1,yt1)
    line = slope*xt1+intercept

    print(slope)
    print(intercept)
    
    trace0 = Scatter(
        x=xt1,
        y=yt1,
        mode='markers',
        name=calc,
    #     marker=dict(

    #     ),
        text=[i[0]+' : '+str(i[1]) for i in df_group[exp].mean().index.get_values()],
        error_y=dict(
            type='data',
            array=yt1e,
            visible=True
        )
    )
    trace2 = Scatter(
                      x=xt1, 
                      y=line, 
                      mode='lines',
                      marker=Marker(color='rgb(31, 119, 180)'),
                      name='Fit'
                      )

    annotation = Annotation(
                        xref='paper',
                        yref='paper',
                       x=0,
                       y=1,
         #xanchor='left',
         #yanchor='top',
                      text='R_value: {:.5f} Slope: {:.4f} Intercept: {:.3f}'.format(r_value,slope,intercept),
                      showarrow=False,
                      font=Font(size=height/40)
                      )

    data = [trace0,trace2]
    layout = Layout(
        xaxis= dict(
            title= exp,
            ticklen= 5,
            zeroline= False,
            gridwidth= 1,
            titlefont=Font(size=height/40),
            tickfont=Font(size=height/40)
        ),
        yaxis=dict(
            title= calc,
            ticklen= 5,
            gridwidth= 1,
            titlefont=Font(size=height/40),
            tickfont=Font(size=height/40)
        ),
                shapes=[dict(
                        type='line',
                        xref='x',
                        yref= 'y',
                        x0= np.min(yt1)-0.05,
                        y0= np.min(yt1)-0.05,
                        x1= np.max(yt1)+0.05,
                        y1= np.max(yt1)+0.05,
                        line = {
                'color': 'red',
                'width': 2,
                'dash':'dash'
            }
                )],
         annotations=[annotation],
        showlegend=False,
        height=height,
        width=width,
    )

    fig = dict( data=data, layout=layout )

    iplot(fig,image_height=height,image_width=width)
    
    #plot(fig,image_height=height,image_width=width,show_link=False,include_plotlyjs=True)


In [ ]:
#plot_calc_vs_theo(df_min,'t1_opt','exp_t1')

In [ ]:
plot_calc_vs_theo_std(df_low,'t1_opt','exp_t1')

In [ ]:
plot_calc_vs_theo_std(df_low,'t1_vert','exp_t1')

In [ ]:
plot_calc_vs_theo_std(df_low,'t1_vert_wb97xd','exp_t1')

In [ ]:
show(df_low[:1])

In [ ]:
def max_min(group, x_n):
    x = group[x_n]
    return np.max(x)-np.min(x)
#['dft_energy']
df_=df_low.reset_index()
#df_['mol__inchikey']=df_['mol__inchikey'].str.split('-').str[0]
df_mae_t1=df_.groupby('mol__inchikey').apply(max_min,'t1_opt').to_frame().rename(columns={0:'mm_t1'})
df_mae_count=df_.groupby('mol__inchikey').count()['geoms__parents'].to_frame().rename(columns={'geoms__parents':'conf_count'})
#print(df_mae_count)
df_mae_s1=df_.groupby('mol__inchikey').apply(max_min,'s1_vert').to_frame().rename(columns={0:'mm_s1'})
df_mae1=df_.set_index('mol__inchikey').groupby(level=0).apply(lambda x: x.sort_values('dft_energy').iloc[0])
#print(df_reaxys1.index)
#print(df_mae1.index)
df_confs=df_mae1.join(df_mae_t1).join(df_mae_s1).join(df_mae_count)#.join(df_smiles)

In [ ]:
show(df_confs[:1])

In [ ]:
#Header for HTML table output
# Reference Semantic.ui
# JQuery
html_start = """
<!DOCTYPE html>
<html>
<head>
<meta http-equiv="Content-type" content="text/html; charset=utf-8">
<meta name="viewport" content="width=device-width,initial-scale=1">
<title>SF Candidates</title>

<link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/v/dt/jqc-1.12.3/jszip-2.5.0/pdfmake-0.1.18/dt-1.10.12/b-1.2.2/b-colvis-1.2.2/b-html5-1.2.2/fh-3.1.2/r-2.1.0/sc-1.4.2/datatables.min.css"/>
 
<script type="text/javascript" src="https://cdn.datatables.net/v/dt/jqc-1.12.3/jszip-2.5.0/pdfmake-0.1.18/dt-1.10.12/b-1.2.2/b-colvis-1.2.2/b-html5-1.2.2/fh-3.1.2/r-2.1.0/sc-1.4.2/datatables.min.js"></script>



<link rel="stylesheet" type="text/css" href="https://cdnjs.cloudflare.com/ajax/libs/semantic-ui/2.2.1/semantic.min.css">
<link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/1.10.12/css/dataTables.semanticui.min.css">
<link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/buttons/1.2.2/css/buttons.semanticui.min.css">


	<script type="text/javascript" language="javascript" src="https://cdn.datatables.net/1.10.12/js/dataTables.semanticui.min.js">
	</script>
	<script type="text/javascript" language="javascript" src="https://cdnjs.cloudflare.com/ajax/libs/semantic-ui/2.2.1/semantic.min.js">
	</script>
    <script type="text/javascript" language="javascript" src="https://cdnjs.cloudflare.com/ajax/libs/3Dmol/1.1.1/3Dmol-nojquery-min.js">
	</script>
	
    <style type="text/css" class="init">
	
	</style>
	<script type="text/javascript" class="init">
	
        $(document).ready(function() {
            var table = $('#example').DataTable( {
                lengthChange: true,
                "lengthMenu": [ 3, 10, 25, 50, 75, 100 ],
                buttons: [ 'copy', 'excel', 'colvis' ],
                
            } );
        
            table.buttons().container()
                .appendTo( $('div.eight.column:eq(0)', table.table().container()) );
        } );
        var blobObject = null;

        function createDownloadLink(anchorSelector, str, fileName){

            if(window.navigator.msSaveOrOpenBlob) {
                var fileData = [str];
                blobObject = new Blob(fileData);
                $(anchorSelector).click(function(){
                    window.navigator.msSaveOrOpenBlob(blobObject, fileName);
                });
            } else {
                var url = "data:text/plain;charset=utf-8," + encodeURIComponent(str);
                $(anchorSelector).attr("href", url);
            }
        }
	</script>
</head>
<body>
<div class="ui container">
"""

html_end = """
</div>
</body>
</html>
"""

#Generate HTML report in interactive table with 3D view

report_file_name="/home/denn/home/ml/data/sf/export_sam/exp.html"

def pop3d(conformer):
    popup = '<div class="ui primary button" id="view3d_{}">View 3D</div>'.format(conformer)
    return popup

def download3d(conformer):
    popup = '<a href="#" class="ui primary button" id="download3d_{}">View 3D</a>'.format(conformer)
    return popup

def buy_emol(tags):
    if 'emol1' in tags:
        return '<i class="shopping bag icon"></i><div style="display: none;">buy</div>'
    else:
        return ''
def pubchem_link(inchikey):
    cell='<a href="https://pubchem.ncbi.nlm.nih.gov/search/#collection=compounds&query_type=text&query={}" target="_blank">{}</a>'.format(inchikey,inchikey)
    return cell

df_html=df_confs.reset_index()
df_html['mol3d']=df_html['mol__inchikey'].apply(pop3d)+'<br>Conformers: '+df_html['conf_count'].map(str)
# df_html['Buy (eMol)']=df_html['mol__tags'].apply(buy_emol)
df_html['mol__inchikey_link']=df_html['mol__inchikey'].apply(pubchem_link)+df_html['smiles']\
    .apply('<div style="width:10em;word-break:break-all;" class="ui small segment" data-tooltip="SMILES could be pasted to ChemDraw or used to search in Reaxys, etc.">{}</div>'.format)
#df_html['T1, eV']=df_html['t1_opt'].apply(lambda s:"{0:.2f}".format(s))+'<br>Max-min: '+df_html['mm_t1'].apply(lambda s:"{0:.2f}".format(s))
html_js = """
    <div class="ui small modal" id="mol3Dmodal">
        <i class="close icon"></i>
        <div class="header">3D view (rotate with mouse)</div>
        <div class="content">
            <div style="height: 350px; width: 720px; position: relative;" class='viewer_3Dmoljs' data-backgroundcolor='0xffffff'></div>
            <div class="action">
                Conformer: <div id="conf_num" class="ui label"></div>
                <div class="ui primary labeled icon button" id="prev_conf" tabindex="0">
                  <i class="left arrow icon"></i>
                  Prev
                </div>
                <div class="ui primary right labeled icon button" id="next_conf" tabindex="0">
                  <i class="right arrow icon"></i>
                  Next
                </div>mi
            </div>

            <p>dft_energy: in eV from lowest energy conformer</p>
            <div id="modal_table"></div>
        </div>
    </div>
    <script>
        $('#conf_num').data('count', 0);
        $('#conf_num').html($('#conf_num').data('count'))
        $('#next_conf').click(function(){
            var count = $('#conf_num').data('count');
            var view = $3Dmol.viewers[0];
            if (count<view.getFrames()-1){
                count = count + 1;
                view.setFrame(count);
                view.zoomTo();
                view.render();
                $('#conf_num').data('count', count);
                $('#conf_num').html(count);
            }
        });
        $('#prev_conf').click(function(){
            var count = $('#conf_num').data('count');
            var view = $3Dmol.viewers[0];
            if (count>0){
                count = count - 1;
                view.setFrame(count);
                view.zoomTo();
                view.render();
                $('#conf_num').data('count', count);
                $('#conf_num').html(count);
            }
        });
    </script>
    <h1 class="ui header">Experimental T1</h1>
    <p class="ui header"></p>
"""
modal_cols=['dft_energy','t1_opt','t1_vert','t2_vert','s1_vert','oscillator_strength_s1']
html_button_js="<script>"
for conformer in df_html['mol__inchikey'][:]:
    html_button_js+="$('#mol3Dmodal').modal('attach events', '#view3d_{}');".format(conformer)
    df_sel=df_low.ix[conformer].sort_values('dft_energy').reset_index()
#     df_sel['down3d']=df_sel.geoms__parents.apply(download3d)
#     print(df_sel.info())
#     print(df_sel['geoms__parents'])
    #print(df_sel.to_html(escape=True).replace('<table border="1" class="dataframe">','<table class="ui celled table" cellspacing="0" width="100%">').replace("\n", "\\n"))
    html_modal_table= "$('#modal_table').html('%s');" % df_sel[modal_cols].to_html(escape=False).replace('<table border="1" class="dataframe">','<table class="ui celled table" cellspacing="0" width="100%">').replace("\n", "")
#     print(html_modal_table)
    #print(df_sel['geoms__parents'].tolist())
    try:
        xyz_str=(''.join(df_xyz.loc[df_sel['geoms__parents']]['xyz'])).replace("\n", "\\n")
    except:
        xyz_str=''.replace("\n", "\\n")
    html_button_js+=("$('#view3d_%s').click(function() { $('#conf_num').data('count', 0); $('#conf_num').html($('#conf_num').data('count')); var view = $3Dmol.viewers[0];view.removeAllModels();view.addModelsAsFrames('%s','xyz');view.setStyle({'stick':{}});view.addStyle({'sphere':{'radius':0.4}});view.zoomTo();view.render();%s});" \
                     % (conformer,xyz_str,html_modal_table))
    
#     j1="$(function () {"
#     j2=""
#     for i,row in df_xyz.loc[df_sel['geoms__parents']].iterrows():
#         j2+='createDownloadLink("#down3d_%s","%s","%s.xyz");' % (i,row['xyz'],i)
#     j3="});"
#     jdown=j1+j2+j3
#     print(jdown)
html_button_js+="</script>"


pd.options.display.float_format = '{:,.2f}'.format
html_table=df_html[['mol__inchikey_link','mol','mol3d','t1_opt','exp_t1','s1_vert','exp_s1']]\
    .rename(index=str, columns={'mol__inchikey_link':'InChIKey linked to PubChem (SMILES in the box)','mol':'Molecule','mol3d':'3D','t1_opt':'T1, eV','s1_vert':'S1, eV','criteria1':'S1-2T1, eV','criteria2':'T2-2T1, eV','mm_t1':'T1 max-min, eV','mm_s1':'S1 max-min, eV'}).to_html(escape=False)
html_table=html_table.replace('<table border="1" class="dataframe">','<table id="example" class="ui celled table" cellspacing="0" width="100%">')

# Write the report to disk 
with open(report_file_name, "w") as text_file:
   text_file.write(html_start+html_js+html_table+html_button_js+html_end)


In [ ]:
df_xyz.sample(1)

In [ ]:
from sklearn.metrics import mean_absolute_error,mean_squared_error
df_=df_min
print(mean_absolute_error(df_['exp_t1'], df_['t1_opt']))
print(np.sqrt(mean_squared_error(df_['exp_t1'], df_['t1_opt'])))

In [ ]:
print(mean_absolute_error(df_['exp_t1'], df_['t1_vert']))
print(np.sqrt(mean_squared_error(df_['exp_t1'], df_['t1_vert'])))

In [ ]:
def mae(group, y_true, y_pred):
    d = group[y_true]
    w = group[y_pred]
    try:
        return np.mean(np.abs((d - w)))
    except ZeroDivisionError:
        return d.mean()
                              
#df_mae=df_min.reset_index().groupby('mol__inchikey').apply(mae,'exp_t1','t1_opt').to_frame()
df_mae=df_min.reset_index().groupby('mol__inchikey').min()#.to_frame()
#df_mae=pd.merge(df_mae, dfexpi, how='outer', right_index=True, left_index=True)
show(df_mae[['mol','s1_vert','s1_vert_lcpbe','s1_vert_wb97xd','exp_s1','t1_opt','exp_t1']])
#show(df_mae)

In [ ]:
# def mm(group, y_true):
#     d = group[y_true]
#     #w = group[y_pred]
#     try:
#         return np.mean(np.abs((d - w)))
#     except ZeroDivisionError:
#         return d.mean()
                              
# df_mae=df_all.reset_index().groupby('mol__inchikey').describe()#.to_frame()
#df_mae=pd.merge(df_mae, dfexpi, how='outer', right_index=True, left_index=True)
#print(df_mae.index)
#show(df_mae[[0,'mol']])
#show(df_mae[['t1_opt','s1_vert']])

from sklearn.metrics import mean_absolute_error
df_=df_min
print(mean_absolute_error(df_['exp_s1'], df_['s1_vert']))
print(mean_absolute_error(df_['exp_s1'], df_['s1_vert']*0.884560153698+0.186701541979))

In [ ]:
plot_calc_vs_theo_std(df_low,'s1_vert','exp_s1')

In [ ]:
plot_calc_vs_theo_std(df_low,'s1_vert_lcpbe','exp_s1')

In [ ]:
plot_calc_vs_theo_std(df_low,'s1_vert_wb97xd','exp_s1')

In [ ]:
plot_calc_vs_theo_std(df_low,'s1_vert_m062x','exp_s1')

In [ ]:
df_mae=df_min.reset_index().groupby('mol__inchikey').apply(mae,'exp_s1','s1_vert').to_frame()
df_mae=pd.merge(df_mae, dfexpi, how='outer', right_index=True, left_index=True)
df_mae.sort_values([0],ascending=False,inplace=True)
#
show(df_mae[[0,'mol']])

In [ ]:
# error_cals = Job.objects.filter(
#             status__exact='error')\
#     .order_by('config__name')
# show(pd.DataFrame(list(error_cals.values('config__name','config__parent_class_name'))))

error_mols = Geom.objects.filter(
            childjobs__status__exact='error')\
            .filter(Q(mol__group__name__exact='sf_lib') )\
            .order_by('mol__inchikey')
dfm=pd.DataFrame(list(error_mols.values('mol__inchikey','mol__smiles','childjobs__config__name','mol__tags')))
dfm['mol']=dfm.mol__smiles.apply(Chem.MolFromSmiles)
#dfm=dfm.set_index(['mol__inchikey','childjobs__config__name'])
show(dfm[['childjobs__config__name','mol','mol__inchikey']])

In [ ]:
#            .filter(Q(mol__group__name__exact='sf_acenes') | Q(mol__group__name__exact='sf'))\

error_mols = Geom.objects.filter(
            childjobs__status__contains='claimed')\
            .order_by('mol__inchikey')

dfm=pd.DataFrame(list(error_mols.values('mol__smiles','childjobs__config__name','mol__tags','childjobs__uuid')))

dfm['mol']=dfm.mol__smiles.apply(Chem.MolFromSmiles)

#show(dfm)

In [ ]:
#            .filter(Q(mol__group__name__exact='sf_acenes') | Q(mol__group__name__exact='sf'))\

emptu_jobs = Geom.objects.filter(
            childjobs__status__contains='claimed')\
            .order_by('mol__inchikey')

dfm=pd.DataFrame(list(error_mols.values('mol__smiles','childjobs__config__name','mol__tags','childjobs__uuid')))

dfm['mol']=dfm.mol__smiles.apply(Chem.MolFromSmiles)

#show(dfm)

In [ ]:
#grouped = df.groupby(by=['mol__inchikey'],axis=0)
#print(grouped.groups)
#print(grouped['parentjob__config__name'].head())

In [ ]:
time_calcs = Calc.objects.filter(mol__group__name__exact='sf', parentjob__status__exact='done',method__name__contains='tddft')\
        .exclude(method__name__exact='tddft_tda_hybrid_b3lyp', parentjob__config__name__exact='b3lyp_6-31gs_tddft')\
    .values('mol__inchikey','parentjob__config__name','parentjob__duration').distinct()\
    .order_by('mol__inchikey')

#print(time_calcs)

dftime = pd.DataFrame(list(time_calcs))

#show(df.head())
#for c in all_calcs:
#    print(c.id,c.mol.inchikey, c.parentjob.config.name, c.parentjob.duration)

In [ ]:
mol = Mol.objects.filter(Q(group__name__exact='sf_acenes') | Q(group__name__exact='sf')).get(inchikey='MMHOLFVKLFQKKD-UHFFFAOYNA-N')

geoms = Geom.objects.filter(mol=mol,parentjob__config__parent_class_name__exact='Mol')

# g = geoms[0]
# 
# g.
for g in geoms[:1]:
    for chg in g.children.all():
        calcs = Calc.objects.filter(parentjob__parentid=chg.id)
        print(calcs)
    
    
#print(geoms.values('method__name'))
#print(mols.geom_set.all().values('calcs__children__parentjob__config__name'))

In [ ]:
jobsformol=Calc.objects.filter(mol__inchikey__exact='MWPLVEDNUUSJAV-UHFFFAOYNA-N').values('parentjob__config__name','parentjob__status','geoms','geoms__parents')
pd.DataFrame(list(jobsformol))

In [ ]:
from django.core import serializers
from django.db.models import Q,F,Min

all_calcs = Calc.objects \
    .filter(Q(mol__group__name__exact='sf_acenes') | Q(mol__group__name__exact='sf'),
            parentjob__status__exact='done', 
            method__name__contains='hybrid') \
    .order_by('mol__inchikey','parentjob__config__name')\
    .exclude(method__name__contains='_tda_')



#print(all_calcs.values_list('parentjob__config__name').distinct())
#print(set(all_calcs.values_list('method__name').distinct())-set(all_calcs.exclude(method__name__contains='_tda_').values_list('method__name').distinct()))
#print()
#    .prefetch_related('props')
#grp_calcs = all_calcs.values('mol__inchikey').annotate(methods = 'parentjob__config__name')
#print(grp_calcs)
#print(serializers.serialize('json',))
#print(all_calcs.values_list('mol__inchikey','mol__smiles','parentjob__config__name','props'))

#for c in all_calcs:
#print(c.geoms.get().as_xyz())
#    print(c.mol.inchikey, c.parentjob.config.name, c.method.name, c.props.get('totalenergy'))
#print(json.dump(list(grp_calcs)))


calc_dict = {}
for key, calc in itertools.groupby(all_calcs, lambda c: c.mol.inchikey):
    listcalc = list(calc)
    energy_dict = dict(map(lambda p: (p.parentjob.config.name, p.props), listcalc))
    mol_dict = dict(map(lambda p: (key, p.mol.smiles), listcalc))
    #print(energy_dict.keys())
    res_dict = {'mol': Chem.MolFromSmiles(mol_dict[key])}
    if 'b3lyp_6-31gs_opt_qchem' in energy_dict:
        if 'b3lyp_6-31gs_opt_s1' in energy_dict:
            s1_opt = energy_dict['b3lyp_6-31gs_opt_s1'].get('totalenergy') * 27.211399 - energy_dict['b3lyp_6-31gs_opt_qchem'].get('totalenergy') * 27.211399
            res_dict.update({'s1_opt': s1_opt})
        if ('b3lyp_6-31gs_opt_t1_qchem' in energy_dict):
            t1_opt = energy_dict['b3lyp_6-31gs_opt_t1_qchem'].get('totalenergy') * 27.211399 - energy_dict[
                                                                                                   'b3lyp_6-31gs_opt_qchem'].get(
                'totalenergy') * 27.211399
            res_dict.update({'t1_opt': t1_opt})
        if ('b3lyp_6-31gs_opt_t1_qchem' in energy_dict) & ('b3lyp_6-31gs_opt_s1' in energy_dict):
            res_dict.update({'s1-2t1': s1_opt - 2 * t1_opt})
    if ('b3lyp_6-31gs_tddft' in energy_dict):
        exst = energy_dict['b3lyp_6-31gs_tddft'].get('excitedstates')
        if exst[0].get('multiplicity') == 'triplet':
            res_dict.update({'t1_vert': (exst[0].get('energy'), exst[0].get('oscillator_strength'))})
        for tran in exst:
            if tran.get('multiplicity') == 'singlet':
                res_dict.update({'s1_vert': (tran.get('energy'), tran.get('oscillator_strength'))})
                break

    if ('lcwpbeh_6-31gs_tddft' in energy_dict):
        exst = energy_dict['lcwpbeh_6-31gs_tddft'].get('excitedstates')
        if exst[0].get('multiplicity') == 'triplet':
            res_dict.update({'t1_vert_lc': (exst[0].get('energy'), exst[0].get('oscillator_strength'))})
        for tran in exst:
            if tran.get('multiplicity') == 'singlet':
                res_dict.update({'s1_vert_lc': (tran.get('energy'), tran.get('oscillator_strength'))})
                break

    if 'b3lyp_6-31gs_opt_qchem' in energy_dict:
        #hl_gap = energy_dict['b3lyp_6-31gs_opt_qchem'].get('lumo')*27.211399-energy_dict['b3lyp_6-31gs_opt_qchem'].get('homo')*27.211399
        hl_gap = (energy_dict['b3lyp_6-31gs_opt_qchem'].get('lumo') - energy_dict['b3lyp_6-31gs_opt_qchem'].get(
            'homo')) * 27.211399
        res_dict.update({'hl_gap': hl_gap})

    method = 'b3lyp_6-31gs_opt_BS_qchem'
    if method in energy_dict:
        hl_gap = (energy_dict[method].get('lumo') - energy_dict[method].get('homo')) * 27.211399
        res_dict.update({'hl_gap_BS': hl_gap})

    calc_dict[key] = res_dict

#print(calc_dict)
df = pd.DataFrame(calc_dict).transpose()

dfexp = pd.read_csv('/home/denn/harvard/SF/Library/exp_set_1_th.csv')
dfexp['block'] = dfexp.smiles.apply(lambda s: block.Block(smiles=s))
dfexp['mol'] = dfexp.block.apply(lambda b: b.mol)
dfexp['inchikey'] = dfexp.block.apply(lambda b: b.inchikey)
dfexpi = dfexp.set_index('inchikey')

df_all = pd.merge(df, dfexpi, on='mol', how='right', right_index=True, left_index=True)

#show(df_all[['mol','hl_gap','hl_gap_BS','s1_vert','s1_vert_lc','t1_opt','exp_s1']][:10])
#show(df[['mol','hl_gap','hl_gap_BS','s1_vert','s1_vert_lc','s1_opt','t1_opt']])
#df.to_html(open('sf_acenes_phz.html', 'w'),escape=False)